# Project — Solar Energy Production Forecasting
## Part 1 — Load data + time-safe Train/Validation split (30%)

Steps:
- Load train and test datasets
- Ensure chronological order (no shuffling)
- Create a 70/30 Train/Validation split **by time**
- Perform minimal sanity checks (shape, missing values, target leakage)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import sys
print(sys.executable)


/opt/anaconda3/envs/Energy-Production-Forecasting/bin/python
